In [2]:
import os
import numpy as np
import pandas as pd
import math
import statistics as stat
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
from id3 import Id3Estimator, export_text
from sklearn.cluster import KMeans
from sklearn import tree
import scipy.spatial.distance as dist
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
warnings.simplefilter(action='ignore', category=(FutureWarning, DeprecationWarning))
missing_values = ['n/a', 'na', 'nan', 'N/A', 'NA', 'NaN', 'NAN', '--', 'Missing', 'missing', 'MISSING', 'Unknown',
                  'Weet ik niet', 'None', 'ONBEKEND', '"ontbrekend"', '"onbekend"', 'onbekend']

In [3]:
def afstand_euclidisch(tabel, i, j):
    rij1 = tabel.iloc[i]
    rij2 = tabel.iloc[j]
    afstand = math.sqrt(((rij1 - rij2) ** 2).sum())
    return round(afstand, 2)


def afstand_manhattan(tabel, i, j):
    rij1 = tabel.iloc[i]
    rij2 = tabel.iloc[j]
    afstand = ((rij1 - rij2).abs()).sum()
    return afstand

# 2 Studenten
1. Lees het bestand "studenten.csv" in. Hierin staan de scores van 100 studenten op bepaalde vakken

In [4]:
df = pd.read_csv('studenten.csv',delimiter=',')
df

,vak1,vak2,vak3,vak4,vak5,vak6,vak7,vak8
0,11,3,13,8,7,13,11,14
1,5,5,10,11,9,11,9,14
2,11,13,16,13,9,9,15,14
3,1,3,5,0,3,4,10,6
4,16,14,12,16,15,20,18,19
5,6,5,8,11,9,16,13,11
6,11,10,15,11,9,12,17,17
7,13,9,16,12,15,10,15,17
8,6,9,6,7,10,9,11,15
9,5,9,9,10,11,11,9,12


2. Bereken de afstand tussen student1 en student2 op de volgende manieren:
     euclidisch
     gestandaardiseerd euclidisch
     manhattan

In [5]:
print(afstand_euclidisch(df, 0, 1))
stanDev = df.std()
exampleZ = pd.DataFrame()
exampleZ[0] = df.iloc[0] / stanDev
exampleZ[1] = df.iloc[1] / stanDev
print(dist.euclidean(exampleZ.iloc[0], exampleZ.iloc[1]))
print(afstand_manhattan(df, 0, 1))

8.37
1.2680908925537264
20


3. We zoeken 4 clusters in deze set. Kies volgende centroids:
     (9, 3, 14, 1, 6, 10, 10, 15)
     (12, 18, 9, 5, 18, 1, 3, 18)
     (6, 15, 13, 18, 9, 15, 20, 18)
     (5, 4, 7, 18, 20, 17, 1, 15)
Bereken nu de euclidische afstand van studenten 1, 10, 20 en 30 tot deze centroids.
4. Bij welke centroid zou je deze studenten indelen?
5. Bepaal de categorieën (clusters) van iedere student a.d.h.v. het K-means algoritme. Hoeveel studenten zitten er in iedere categorie?
6. Voeg een kolom toe waarin de categorie staat. Gebruik DecisionTreeClassifier in Python om een beslissingsboom op te stellen.
7. Hoe zou je deze 4 categorieën typeren?

In [7]:
centoids = np.array([[9, 3, 14, 1, 6, 10, 10, 15],[12, 18, 9, 5, 18, 1, 3, 18],[6, 15, 13, 18, 9, 15, 20, 18],[5, 4, 7, 18, 20, 17, 1, 15]])

kmeans = KMeans(n_clusters=4, init=centoids, n_init=1)
kmeans.fit(df)

dflbl = kmeans.labels_


8. In welke categorie valt een nieuwe student met de volgende punten: (10, 15, 12, 11, 13, 14, 9, 10)?

In [8]:
kmeans.predict([[10, 15, 12, 11, 13, 14, 9, 10]])

H:\My Drive\School\2022-2023\Data-Science\Data-Science\venv\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


array([0])